In [ ]:
import os
import caselawnet
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import json
import networkx as nx
import community
import nwtools.communities

In [ ]:
filepath = '/media/sf_VBox_Shared/CaseLaw/2018-01-29-lido/derived/'
nodes_df = pd.read_csv(os.path.join(filepath, 'case_nodes_simple_bimodal.csv'), index_col='lido_id')
edges_df = pd.read_csv(os.path.join(filepath, 'case_to_case_lx_links.csv'))

In [ ]:
partition = community.best_partition(graph.to_undirected())
nodes_df['community'] = pd.Series(partition)

In [ ]:
edges_df.head()

In [ ]:
nodes_df.head()

In [ ]:
graph = nx.from_pandas_edgelist(edges_df, source='source', target='target', create_using=nx.DiGraph())
attributes = ['country', 'court', 'year', 'community', 'community_bimodal']
for att in attributes:
    nx.set_node_attributes(graph, nodes_df[att].to_dict(), att)

## Assortativity

In [ ]:
for att in attributes[1:]:
    ass = 0 #nx.attribute_assortativity_coefficient(graph, att)
    mod = community.modularity(nodes_df[att].to_dict(), graph.to_undirected())
    print(att, 'assortativity:', ass, 'modularity:', mod)

In [ ]:
nx.adjacency_matrix?

## Degree distributions

In [ ]:
degree_hist = nx.degree_histogram(graph)

In [ ]:
indegrees = list(dict(graph.in_degree()).values())
outdegrees = list(dict(graph.out_degree()).values())

In [ ]:
len(degree_hist)

In [ ]:
fig, axes = plt.subplots(3, 1, sharex=True, figsize=(5, 15))
axes[0].hist(indegrees, bins=range(max(indegrees)))
axes[0].set_xscale("log")
axes[0].set_yscale("log")
axes[0].set_title('In-degree')

axes[1].hist(outdegrees,  bins=range(max(outdegrees)))
axes[1].set_xscale("log")
axes[1].set_yscale("log")
axes[1].set_title('Out-degree')

n = 500
axes[2].bar(range(500), degree_hist[:n])
axes[2].set_xscale("log")
axes[2].set_yscale("log")
axes[2].set_title('Degree')
plt.show()

## Connected components

In [ ]:
import numpy as np

In [ ]:
ccs = list(nx.connected_components(graph.to_undirected()))

In [ ]:
ccs_multiple = [c for c in ccs if len(c) > 1]
ccs_sizes = np.array([len(c) for c in ccs])
ccs_multiple_sizes = np.array([len(c) for c in ccs_multiple])
print("Number of connected components:", len(ccs))
print("Relative size of largest component:", np.max(ccs_sizes)/np.sum(ccs_sizes))
print("Number of non-singleton components:", len(ccs_multiple_sizes))
print("Relative size of largest component without singletons:", np.max(ccs_multiple_sizes)/np.sum(ccs_multiple_sizes))

In [ ]:
# Save largest cc
gcc_ids = list(ccs[np.argmax(ccs_sizes)])
#nodes_gcc = nodes_df.loc[gcc_ids]
edges_gcc = edges_df[edges_df['source'].isin(gcc_ids) & edges_df['target'].isin(gcc_ids)]
#print(len(nodes_gcc), len(edges_gcc))
#nodes_gcc.to_csv(os.path.join(filepath, 'node__gcc.csv'))
edges_gcc.to_csv(os.path.join(filepath, 'case_to_case_links_lx_gcc.csv'), index=False)

In [ ]:
edges_gcc.shape

## communities

In [ ]:
community_sizes = nodes_df.groupby('community').size()
print("Number of communities:", len(community_sizes))
print("Average size of community", community_sizes.mean())